In [1]:
import os.path
import glob
import Input.Input
from Sample import Sample
import cPickle as pickle
import subprocess
import os
import shutil

In [ ]:
# script for converting native instuments stem format mp4 files into wavs using stem2wav
# creates a directory for each filename and places the 5 separated wav files in directory
# requires stempeg ---> 'pip install stempeg'


path = # '~/data/nistems/acc'
files = [name for name in glob.glob(path + '/*')]
print(files)
for file in files:
    newdir = file[31:-9]
    print(newdir)
    newpath = path + newdir + file[31:]
    print(newpath)
    if not os.path.exists(path + newdir):
        os.makedirs(path + newdir)
        subprocess.call('stem2wav ' + file + ' ' + path + newdir, shell=True)

In [ ]:

# Mixes together accompaniment of a stem track into acc.wav for unsup_acc use from original
# NI Stems processed with stem2wav and renamed to mix.wav, drums.wav, acc1.wav, acc2.wav, acc3.wav
# SETUP: each track must have a directory containing all five wav files as subdirectories of path
# named as above
# old acc files and the drums and mix files are removed if successful
# requires ffmpeg

path = '/Users/dfreezy/data/nistems/acc_only'
folders = [name for name in glob.glob(path + '/*')]
print(files)
for folder in folders:
    fullpath = path + folder
    subprocess.call(
        'ffmpeg -i ' + folder + '/acc1.wav -i ' + folder + '/acc2.wav -i ' + folder + \
        '/acc3.wav -filter_complex amix=inputs=3:duration=longest -ac 2 ' + folder + '/acc.wav', shell=True)
    if os.path.exists(folder + '/acc.wav'):
        subprocess.call('rm ' + folder + '/acc1.wav ' + folder + '/acc2.wav ' + folder + '/acc3.wav ' \
                        + folder + '/drums.wav ' + folder + '/mix.wav', shell=True)
    else:
        print('File acc.wav was not created in ' + path)

    
    

In [ ]:

# process nistems for sup
# Mixes together accompaniment of a stem track into drums, mix, and acc from original
# NI Stems processed with stem2wav and renamed to mix.wav, drums.wav, acc1.wav, acc2.wav, acc3.wav
# SETUP: each track must have a directory containing all five wav files as subdirectories of path
# named as above
# old files are removed if successful
# requires ffmpeg

path = ''       #data/musdb18/test'
folders = [name for name in glob.glob(path + '/*')]
print(folders)
for folder in folders:
    print(folder)
    # folder = folder.replace(" ", "\\ ")  # use for spaces in path
    subprocess.call(
        'ffmpeg -i ' + folder + '/acc1.wav -i ' + folder + '/acc2.wav -i ' + folder + \
        '/acc3.wav -filter_complex amix=inputs=3:duration=longest -ac 2 ' + folder + '/acc.wav', shell=True)
    if os.path.exists(folder + '/acc.wav'):
        subprocess.call('rm ' + folder + '/acc1.wav ' + folder + '/acc2.wav ' + folder + '/acc3.wav', shell=True)
    else:
        print('File acc.wav was not created in ' + path)

    

In [12]:
#get list of paths to wav files. musdb18 DB must already be converted using MUSDB docker tool
# data should be in this format:
# ├── test     # one folder for each track with 3 files mix.wav, drums.wav, and acc.wav
# ├── train    # one folder for each track with 3 files mix.wav, drums.wav, and acc.wav
# ├── valid    # one folder for each track with 3 files mix.wav, drums.wav, and acc.wav
# └── train_unsup   # three folders unsup_acc, unsup_drums, unsup_mix. each with numbered
                    # files, ex. acc_1.wav, mix_34.wav, ect in the respective folders

path = '/Users/ubuntu/data' #set to musdb18 path
train_paths = [name for name in glob.glob(path + '/train/*')]
test_paths = [name for name in glob.glob(path + '/test/*')]
valid_paths = [name for name in glob.glob(path + '/valid/*')]
unsup_drums_paths = [name for name in glob.glob(path + '/train_unsup/unsup_drums/*')]
unsup_mix_paths = [name for name in glob.glob(path + '/train_unsup/unsup_mix/*')]
unsup_acc_paths = [name for name in glob.glob(path + '/train_unsup/unsup_acc/*')]


['/Users/dfreezy/data/musdb18/train/The Districts - Vermont', '/Users/dfreezy/data/musdb18/train/BigTroubles - Phantom', '/Users/dfreezy/data/musdb18/train/Music Delta - Rockabilly', '/Users/dfreezy/data/musdb18/train/Johnny Lokke - Promises & Lies', '/Users/dfreezy/data/musdb18/train/Hezekiah Jones - Borrowed Heart', '/Users/dfreezy/data/musdb18/train/Steven Clark - Bounty', '/Users/dfreezy/data/musdb18/train/Young Griffo - Pennies', "/Users/dfreezy/data/musdb18/train/Phre The Eon - Everybody's Falling Apart", '/Users/dfreezy/data/musdb18/train/Music Delta - Gospel', '/Users/dfreezy/data/musdb18/train/Remember December - C U Next Time', '/Users/dfreezy/data/musdb18/train/Actions - South Of The Water', '/Users/dfreezy/data/musdb18/train/Jokers, Jacks & Kings - Sea Of Leaves', '/Users/dfreezy/data/musdb18/train/Alexander Ross - Velvet Curtain', '/Users/dfreezy/data/musdb18/train/Alexander Ross - Goodbye Bolero', '/Users/dfreezy/data/musdb18/train/Black Bloc - If You Want Success', '/Use

In [8]:
dataset = dict()
dataset["train_sup"] = [] # [list of tuples containing sample oblects (mix, acc, drums)]
dataset["train_unsup"] = [] # Initialise unsupervised dataset [[mix_samples], [acc_samples], [drums]]
dataset["valid"] = [] # list of tuples containing sample oblects (mix, acc, drums)
dataset["test"] = [] # list of tuples containing sample oblects (mix, acc, drums)

test


In [ ]:
# create dataset from filename lists using Sample objects
# add_audio() uses librosa which requires python 2.7
for path in train_paths:
    mix = Sample.from_path(path + '/mix.wav')
    drums = Sample.from_path(path + '/drums.wav')
    acc = Sample.from_path(path + '/acc.wav')
    dataset['train_sup'].append((mix, acc, drums))

for path in test_paths:
    mix = Sample.from_path(path + '/mix.wav')
    drums = Sample.from_path(path + '/drums.wav')
    acc = Sample.from_path(path + '/acc.wav')
    dataset['test'].append((mix, acc, drums))
    
for path in valid_paths:
    mix = Sample.from_path(path + '/mix.wav')
    drums = Sample.from_path(path + '/drums.wav')
    acc = Sample.from_path(path + '/acc.wav')
    dataset['valid'].append((mix, acc, drums))

In [ ]:
mix = [Sample.from_path(path + '/mix.wav') for path in unsup_mix_paths]
acc = [Sample.from_path(path + '/acc.wav') for path in unsup_acc_paths]
drums = [Sample.from_path(path + '/drums.wav') for path in unsup_drums_paths]
dataset['train_unsup'] = [mix, acc, drums]

    
    


'/Users/dfreezy/DSSGAN'

In [5]:
with open('dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)
print("Created dataset structure")
print('test: ' + str(len(dataset['test'])
print('valid: ' + str(len(dataset['valid'])
print('train_sup: ' + str(len(dataset['train_sup'])
print('train_unsup_mix: ' + str(len(dataset['train_unsup'][0])
print('train_unsup_acc: ' + str(len(dataset['train_unsup'][1])
print('train_unsup_drums: ' + str(len(dataset['train_unsup'][2])

In [ ]:
dataset['train_sup'][0][0].path
dataset['train_unsup'][0][0].path
dataset['train_unsup'][1][-1].path